In [2]:
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

from aesvdd import autoencoder
from aesvdd import model
from aesvdd import datasets

import warnings
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
from tensorflow.keras import Model
from tensorflow import keras
import pandas as pd
import numpy as np

import time
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
print(os.getcwd())
warnings.filterwarnings("ignore")


/home/kael99/Documents/Code/Autoencoder_SVDD


In [11]:
# X, y = datasets.get_paysim()
# X.tofile("X.dat")
# y.tofile("y.dat")
X = np.fromfile("X.dat").reshape(-1,9)
y = np.fromfile("y.dat", dtype=np.int64)
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=10000, test_size=5000)


In [12]:
tf.reset_default_graph()
ae_model = autoencoder.VAE(z_dim=5)

In [13]:
# pretrain
ae_model.fit(X_train, shuffle=True, epochs=5, batch_size=100, validation_data=(X_test, None), verbose=1)


Train on 300000 samples, validate on 50000 samples
Epoch 1/5
300000/300000 [==============================] - 7s 22us/sample - loss: 0.5509 - val_loss: -0.1928
Epoch 2/5
300000/300000 [==============================] - 6s 19us/sample - loss: -0.1883 - val_loss: -0.1931
Epoch 3/5
300000/300000 [==============================] - 6s 21us/sample - loss: -0.1883 - val_loss: -0.1934
Epoch 4/5
300000/300000 [==============================] - 7s 22us/sample - loss: -0.1885 - val_loss: -0.1930
Epoch 5/5
300000/300000 [==============================] - 6s 21us/sample - loss: -0.1888 - val_loss: -0.1932


In [14]:
inputs = ae_model.input
outputs = ae_model.get_layer(name="code_layer").output
encoder = Model(inputs=inputs, outputs=outputs)

In [15]:
ae_svdd = model.AESVDD(encoder, input_shape=(9,), representation_dim=5, objective="hard", nu=0.05, lr=2e-4)

In [16]:
t0 = time.time()
hisory = ae_svdd.fit(X_train, X_test, y_test, epochs=10, verbose=True)
print('Train time:', time.time() - t0)

# test DeepSVDD
t0 = time.time()
score = ae_svdd.predict(X_test)
auc = roc_auc_score(y_test, -score)
print('Test time:', time.time() - t0)
print('------------- AUROC ----------- : %.4f' % auc)


  5%|▌         | 119/2344 [00:00<00:01, 1181.22it/s]

Epoch:   0 AUROC: 0.714


  5%|▌         | 121/2344 [00:00<00:01, 1205.91it/s]

Epoch:   1 AUROC: 0.812


  6%|▌         | 132/2344 [00:00<00:01, 1319.48it/s]

...

Epoch:   7 AUROC: 0.924


  6%|▌         | 143/2344 [00:00<00:01, 1423.37it/s]

Epoch:   8 AUROC: 0.899


100%|██████████| 2344/2344 [00:01<00:00, 1290.34it/s]


Epoch:   9 AUROC: 0.814
Train time: 22.321998357772827
Test time: 0.18097996711730957
------------- AUROC ----------- : 0.8137
